In [2]:
from keras.layers import Dense,Conv2D,Dropout,Flatten,MaxPooling2D
from keras.preprocessing.image import load_img
from tensorflow.keras.utils import load_img
from keras.utils import to_categorical
from keras.models import Sequential
import pandas as pd
import numpy as np
import os

In [3]:
TRAIN_DIR = 'E:\Data_Sets\images\\train'
TEST_DIR = 'E:\Data_Sets\images\\test'

In [4]:
def createdataframe(dir):
    image_paths=[]
    labels=[]
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label,"completed")
    return image_paths,labels

In [5]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)
# 16:00

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [6]:
print(train)

                                             image     label
0            E:\Data_Sets\images\train\angry\0.jpg     angry
1            E:\Data_Sets\images\train\angry\1.jpg     angry
2           E:\Data_Sets\images\train\angry\10.jpg     angry
3        E:\Data_Sets\images\train\angry\10002.jpg     angry
4        E:\Data_Sets\images\train\angry\10016.jpg     angry
...                                            ...       ...
28816  E:\Data_Sets\images\train\surprise\9969.jpg  surprise
28817  E:\Data_Sets\images\train\surprise\9985.jpg  surprise
28818  E:\Data_Sets\images\train\surprise\9990.jpg  surprise
28819  E:\Data_Sets\images\train\surprise\9992.jpg  surprise
28820  E:\Data_Sets\images\train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]


In [7]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)


angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [8]:
print(test)

                                           image     label
0       E:\Data_Sets\images\test\angry\10052.jpg     angry
1       E:\Data_Sets\images\test\angry\10065.jpg     angry
2       E:\Data_Sets\images\test\angry\10079.jpg     angry
3       E:\Data_Sets\images\test\angry\10095.jpg     angry
4       E:\Data_Sets\images\test\angry\10121.jpg     angry
...                                          ...       ...
7061  E:\Data_Sets\images\test\surprise\9806.jpg  surprise
7062  E:\Data_Sets\images\test\surprise\9830.jpg  surprise
7063  E:\Data_Sets\images\test\surprise\9853.jpg  surprise
7064  E:\Data_Sets\images\test\surprise\9878.jpg  surprise
7065   E:\Data_Sets\images\test\surprise\993.jpg  surprise

[7066 rows x 2 columns]


In [9]:
from tqdm.notebook import tqdm

In [10]:
def extract_features(images):
    features=[]
    for image in tqdm(images):
        img = load_img(image,grayscale = True)
        img = np.array(img)
        features.append(img)
    features=np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features

In [11]:
train_features= extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

c:\Python311\Lib\site-packages\keras\src\utils\image_utils.py:409: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn(


In [12]:
test_features= extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [13]:
x_train = train_features/255.0
x_test = test_features/255.0

In [14]:
from sklearn.preprocessing import LabelEncoder

In [15]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [16]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [17]:
y_train = to_categorical(y_train,num_classes=7)
y_test = to_categorical(y_test,num_classes=7)

In [18]:
model = Sequential()
# 
model.add(Conv2D(128,kernel_size=(3,3), activation='relu',input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256,kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512,kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512,kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# 
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.3))
# 
model.add(Dense(7, activation='softmax'))


In [19]:
# model.compile(optimizer='adam', loss='categorial_crossentropy', metrics = 'accuracy')

In [25]:
# Correct the loss function name
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Now you can use model.fit with the correct loss function
history = model.fit(x=x_train, y=y_train, batch_size=128, epochs=2, validation_data=(x_test, y_test))

# model.fit(x= x_train,y= y_train, batch_size= 128, epochs=100, validation_data =(x_test,y_test))

Epoch 1/2
 39/226 [====>.........................] - ETA: 5:49 - loss: 1.6508 - accuracy: 0.3454

KeyboardInterrupt: 